# Imports

In [10]:
import os
import random
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
import torch.autograd as autograd
from torch.autograd import Variable
from collections import deque, namedtuple

# Building the AI
## Architecture of the network

In [11]:
class Network(nn.Module):

    # Initialization of network architecture
    def __init__(self, state_size, action_size, seed=42):
        super(Network, self).__init__()
        # Random seed
        self.seed = torch.manual_seed(seed)
        # Three layer MLP
        self.fc1 = nn.Linear(state_size, 64)
        self.fc2 = nn.Linear(64, 64)
        self.fc3 = nn.Linear(64, action_size)

    # Forward Pass
    def forward(self, state):
        # First layer with ReLU activation
        x = self.fc1(state)
        x = F.relu(x)
        # Second layer with ReLU activation
        x = self.fc2(x)
        x = F.relu(x)
        # Third layer output
        return self.fc3(x)

# Training the AI

In [12]:
# Environment setup
import gymnasium as gym
env = gym.make('LunarLander-v2')
state_shape = env.observation_space.shape
state_size = env.observation_space.shape[0]
number_actions = env.action_space.n

In [13]:
print(f"State Shape: {state_shape}\nState Size: {state_size}\nNumber of Actions: {number_actions}")

State Shape: (8,)
State Size: 8
Number of Actions: 4


In [14]:
# Hyperparameter Initialization
lr = 5e-4
minibatch_size = 100
gamma = 0.99
replay_buffer_size = int(1e5)
tau = 1e-3

In [15]:
# Experience Replay
class ReplayMemory(object):

    # Initialization
    def __init__(self, capacity):
        self.device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
        self.capacity = capacity
        self.memory = []

    # Adds experience to memory buffer
    def push(self, event):
        self.memory.append(event)
        if len(self.memory) > self.capacity:
            del self.memory[0]

    # Randomly selects batch of experiences from memory buffer
    def sample(self, batch_size):
        experiences = random.sample(self.memory, k=batch_size)
        states = torch.from_numpy(np.vstack([e[0] for e in experiences if e is not None])).float().to(self.device)
        actions = torch.from_numpy(np.vstack([e[1] for e in experiences if e is not None])).long().to(self.device)
        rewards = torch.from_numpy(np.vstack([e[2] for e in experiences if e is not None])).float().to(self.device)
        next_states = torch.from_numpy(np.vstack([e[3] for e in experiences if e is not None])).float().to(self.device)
        dones = torch.from_numpy(np.vstack([e[4] for e in experiences if e is not None]).astype(np.uint8)).float().to(self.device)
        return states, next_states, actions, rewards, dones

In [16]:
class Agent():

    # Initialization
    def __init__(self, state_size, action_size):
        self.device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
        self.state_size = state_size
        self.action_size = action_size
        self.local_qnetwork = Network(state_size, action_size).to(self.device)
        self.target_qnetwork = Network(state_size, action_size).to(self.device)
        # Adam class takes local_qnet weights and learning rate
        self.optimizer = optim.Adam(self.local_qnetwork.parameters(), lr=lr)
        self.memory = ReplayMemory(replay_buffer_size)
        self.t_step = 0

    # Store experiences
    def step(self, state, action, reward, next_state, done):
        self.memory.push((state, action, reward, next_state, done))
        self.t_step = (self.t_step + 1) % 4
        if self.t_step == 0:
            if len(self.memory.memory) > minibatch_size:
                experiences = self.memory.sample(minibatch_size)
                self.learn(experiences, gamma)

    # Select an action
    def act(self, state, epsilon=0.):
        # Add batch dimension to state
        state = torch.from_numpy(state).float().unsqueeze(0).to(self.device)
        # Set to eval mode
        self.local_qnetwork.eval()
        # Does not save gradients
        with torch.no_grad():
            action_values = self.local_qnetwork(state)
        # Reset to training mode
        self.local_qnetwork.train()
        # Select best action 1-epsilon of the time (don't explore)
        if random.random() > epsilon:
            return np.argmax(action_values.cpu().data.numpy())
        # Select random action epsilon of the time (explore)
        else:
            return random.choice(np.arange(self.action_size))

    # Update Agent Q values
    def learn(self, experiences, gamma):
        states, next_states, actions, rewards, dones = experiences
        next_q_targets = self.target_qnetwork(next_states).detach().max(1)[0].unsqueeze(1)
        q_targets = rewards + (gamma * next_q_targets * (1 - dones))
        q_expected = self.local_qnetwork(states).gather(1, actions)
        loss = F.mse_loss(q_expected, q_targets)
        self.optimizer.zero_grad()
        loss.backward()
        self.optimizer.step()
        self.soft_update(self.local_qnetwork, self.target_qnetwork, tau)

    # Update target parameters
    def soft_update(self, local_model, target_model, tau):
        for target_param, local_param in zip(target_model.parameters(), local_model.parameters()):
            target_param.data.copy_(tau * local_param.data + (1.0 - tau) * target_param.data)

In [17]:
# Initialize an Agent
agent = Agent(state_size, number_actions)

In [18]:
# Training hyperparameters
number_episodes = 2000
maximum_number_timesteps_per_episode = 1000
epsilon_starting_value = 1.0
epsilon_ending_value = 0.01
epsilon_decay_value = 0.995
epsilon = epsilon_starting_value
scores_on_100_episodes = deque(maxlen=minibatch_size)

# Train the DQN Agent
for episode in range(1, number_episodes + 1):
    state, _ = env.reset()
    score = 0
    for t in range(maximum_number_timesteps_per_episode):
        action = agent.act(state, epsilon)
        next_state, reward, done, _, _, = env.step(action)
        agent.step(state, action, reward, next_state, done)
        state = next_state
        score += reward
        if done:
            break
    scores_on_100_episodes.append(score)
    epsilon = max(epsilon_ending_value, epsilon_decay_value * epsilon)
    # Dynamic print
    print(f'\rEpisode {episode}\tAverage Score: {np.mean(scores_on_100_episodes):.2f}', end="")
    if episode % 100 == 0:
        print(f'\rEpisode {episode}\tAverage Score: {np.mean(scores_on_100_episodes):.2f}')
    if np.mean(scores_on_100_episodes) >= 200.0:
        print(f'\nEnvironment solved in {episode - 100}!\tEpisode {episode}\tAverage Score: {np.mean(scores_on_100_episodes):.2f}')
        torch.save(agent.local_qnetwork.state_dict(), 'checkpoint.pth')
        break

Episode 100	Average Score: -163.56
Episode 200	Average Score: -121.35
Episode 300	Average Score: -56.762
Episode 400	Average Score: -27.70
Episode 500	Average Score: -1.193
Episode 600	Average Score: 123.97
Episode 666	Average Score: 201.82
Environment solved in 566!	Episode 666	Average Score: 201.82


In [24]:
# Visualize the results
import glob
import io
import base64
import imageio
from IPython.display import HTML, display
from gymnasium.wrappers.monitoring.video_recorder import VideoRecorder

def show_video_of_model(agent, env_name):
    env = gym.make(env_name, render_mode='rgb_array')
    state, _ = env.reset()
    done = False
    frames = []
    while not done:
        frame = env.render()
        frames.append(frame)
        action = agent.act(state)
        state, reward, done, _, _ = env.step(action.item())
    env.close()
    imageio.mimsave('video.mp4', frames, fps=30)

show_video_of_model(agent, 'LunarLander-v2')

def show_video():
    mp4list = glob.glob('*.mp4')
    if len(mp4list) > 0:
        mp4 = mp4list[0]
        video = io.open(mp4, 'r+b').read()
        encoded = base64.b64encode(video)
        display(HTML(data='''<video alt="test" autoplay
                loop controls style="height: 400px;">
                <source src="data:video/mp4;base64,{0}" type="video/mp4" />
             </video>'''.format(encoded.decode('ascii'))))
    else:
        print("Could not find video")

show_video()

IMAGEIO FFMPEG_WRITER WARNING: input image is not divisible by macro_block_size=16, resizing from (600, 400) to (608, 400) to ensure video compatibility with most codecs and players. To prevent resizing, make your input image divisible by the macro_block_size or set the macro_block_size to 1 (risking incompatibility).
[swscaler @ 0x5cd4300] Warning: data is not aligned! This can lead to a speed loss
